<a href="https://colab.research.google.com/github/EdoGiordani99/Traffic-Light-Detector/blob/main/Training_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Object Detection Training Notebook (from scratch)
<h2>Vision & Perception Project 2021/2022</h2>
<h3>Student: Edoardo Giordani</h3>
<h3>E-mail: giordani.2020434@studenti.uniroma1.it</h3>

This notebook was used to train my from scratch implementation of a Object Detection Network. I took inspiration from the Faster RCNN architecture which turned out to be one of the most popular approach of solving the object detection problem. 

##Mounting Drive
This will allow you to save the plots and models in your drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/V&P/TLD

Mounted at /content/drive
/content/drive/MyDrive/V&P/TLD


## Install Dependencies & Import Libraries
As first thing, we install the dependencies and we import the libraries.

In [ ]:
# REQUIREMENTS
!pip install roboflow
!pip install albumentations==0.4.6
!pip install pytest-shutil

In [ ]:
import os
import torch
import pickle
import shutil
import glob as glob
import albumentations
from roboflow import Roboflow
from albumentations.pytorch import ToTensorV2

## Cloning our own Github Repository
Here we are cloning my git repository. This will import in the environment all the code and funcions to train the network and process the data.

In [ ]:
!git clone https://github.com/EdoGiordani99/Traffic-Light-Detector.git

fatal: destination path 'Traffic-Light-Detector' already exists and is not an empty directory.


#### Reorganizing the Folders
(just to make things more clear)

In [ ]:
# Initialiting the output directory
!mkdir outputs

# Transfering .py files to main directory
for filename in os.listdir('Traffic-Light-Detector'):
    if filename != '.git' and filename != 'README.md': 
        old = 'Traffic-Light-Detector/'+filename
        new = filename

        shutil.move(old, new)

# Remooving the github repository folder
!rm -rf Traffic-Light-Detector

mkdir: cannot create directory ‘outputs’: File exists


## Download the Dataset
The dataset was manually collected by myself: I shooted some photos with an high resolution camera of traffic lights all over the city. The images were then processed with the help of the **RoboFlow** framework. This made the annotation and labeling process really easy and fast. 

Here we are simply downloading the annotated dataset from my Roboflow Account

In [ ]:
rf = Roboflow(api_key="cJ4ImlUPhvj3p4yBOaJj")
project = rf.workspace("traffic-light-detector-bnryi").project("traffic-light-detector-o3g9q")
dataset = project.version(2).download("voc")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Traffic-Light-Detector-2 in voc:: 100%|██████████| 413/413 [00:14<00:00, 29.22it/s]


In [ ]:
os.rename('Traffic-Light-Detector-2', 'data')

## Training
Finally, we get to the training. All the hyper-parameters can be set into the *config.py* file.


Unfortunately, due to Colab limitations, I didn't manage to carry on with very long training (over 50 epochs). Epochs could be increased by reducing image size: in this way training will be faster since we have less parameters to learn.

In [ ]:
#@title Run only if a previous training was done
!rm outputs/valid_history
!rm outputs/train_history

rm: cannot remove 'outputs/valid_history': No such file or directory
rm: cannot remove 'outputs/train_history': No such file or directory


In [ ]:
!python train.py

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Number of training samples: 143
Number of validation samples: 35

Setting the Adam optimizer

EPOCH 1 of 50
Loss: 0.4117: 100% 18/18 [00:57<00:00,  3.17s/it]
Validating
Loss: 0.3998: 100% 5/5 [00:18<00:00,  3.60s/it]
Epoch #1 train loss: 1.560
Epoch #1 validation loss: 0.586
Took 1.254 minutes for epoch 0

Best validation loss: 0.5863295555114746

Saving best model for epoch: 1

LOSS PLOTS HAVE BEEN SAVED!

EPOCH 2 of 50
Loss: 0.3596: 100% 18/18 [01:03<00:00,  3.51s/it]
Validating
Loss: 0.4275: 100% 5/5 [00:19<00:00,  3.94s/it]
Epoch #2 t

##Save the model and histories into the drive

In [ ]:
# Copying histories and trained models to make comparisons
!cp outputs/train_history trainings/7/
!cp outputs/valid_history trainings/7/
!cp outputs/best_model.pth trainings/7/
!cp outputs/last_model.pth trainings/7/

## Train-val loss Preview

In [ ]:
with open('outputs/train_history', 'rb') as f:
    train_history = pickle.load(f)

with open('outputs/valid_history', 'rb') as f:
    valid_history = pickle.load(f)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
fig = plt.figure()
ax = plt.axes()

x = np.linspace(0, len(train_history), len(train_history))
train_line = ax.plot(x, train_history);
valid_line = ax.plot(x, valid_history);
ax.legend(['train loss', 'valid loss'])
 
ax.ylim([1, 0])